In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import syft as sy 
import random 

In [15]:
class Arguments():
    def __init__(self, epochs=1):
        self.batch_size = 128
        self.test_batch_size = 1000
        self.epochs = epochs
        self.lr = 0.01
        self.momentum = 0.5
        self.no_cuda = True
        self.seed = 200226097 ## TODO change seed to your studentID inside the class Arguments (line 17)
        self.log_interval = 30
        self.save_model = False

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)


def train(args, model, device, federated_train_loader, optimizer, epoch, participates):
    model.train()  # <-- initial training
    for batch_idx, (data, target) in enumerate(federated_train_loader): # <-- now it is a distributed dataset
        if target.location.id in participates:
            model.send(data.location) # <-- NEW: send the model to the right location
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = F.nll_loss(output, target)
            loss.backward()
            optimizer.step()
            model.get() # <-- NEW: get the model back
            if batch_idx % args.log_interval == 0:
                loss = loss.get() # <-- NEW: get the loss back
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * args.batch_size, len(federated_train_loader) * args.batch_size,
                    100. * batch_idx / len(federated_train_loader), loss.item()))


            
def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


### main function
def run(N=1, X=1):
    args = Arguments(epochs = N)
    use_cuda = not args.no_cuda and torch.cuda.is_available()
    torch.manual_seed(args.seed) 
    device = torch.device("cuda" if use_cuda else "cpu")
    kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

    hook = sy.TorchHook(torch)  # <-- NEW: hook PyTorch ie add extra functionalities to support Federated Learning

    ## TODO: ----  create 10 node workers  ---- ##
    worker_ids = []
    for i in range(10):
        worker_name = f"worker{i}"
        worker = sy.VirtualWorker(hook, id=worker_name)
        worker_ids.append(worker_name)

    ##-------------------------------------------

    ## distribute data across nodes
    federated_train_loader = sy.FederatedDataLoader( # <-- this is now a FederatedDataLoader 
        datasets.MNIST('./data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ]))
        .federate((workers)), ##TODO: pass the worker nodes you created here to distribute the data
        batch_size=args.batch_size, shuffle=True, **kwargs)

    ## test dataset is always same at the central server
    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('./data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=args.test_batch_size, shuffle=True, **kwargs)

    ## training models in a federated appraoch
    model = Net().to(device)
    optimizer = optim.SGD(model.parameters(), lr=args.lr) 

    ## TODO: select a random set of node ids that will be passed to the training function; these nodes will particiapte in the federated learning
    #create node_list 

    node_list = random.sample(worker_ids, k=X)

    ##-------------------------------------------

    for epoch in range(1, args.epochs + 1):
        train(args, model, device, federated_train_loader, optimizer, epoch, node_list ) ## TODO: pass the node_id list like ['node1','node2' ...]
        test(args, model, device, test_loader)

    if (args.save_model):
        torch.save(model.state_dict(), "mnist_cnn.pt")








In [16]:
print("Problem 2a.")
x_vals = [3, 5, 7, 10]
for x in x_vals:
    print(f"Running with N=3, X={x}")
    run(N=3, X=x)

    

Problem 2a.
Running with N=3, X=3
Train Epoch: 1 [30720/60032 (51%)]	Loss: 2.298272
Train Epoch: 1 [34560/60032 (58%)]	Loss: 2.150344
Train Epoch: 1 [49920/60032 (83%)]	Loss: 1.947209
Train Epoch: 1 [53760/60032 (90%)]	Loss: 1.381586
Train Epoch: 1 [57600/60032 (96%)]	Loss: 0.765479

Test set: Average loss: 0.6228, Accuracy: 8326/10000 (83%)

Train Epoch: 2 [30720/60032 (51%)]	Loss: 0.588485
Train Epoch: 2 [34560/60032 (58%)]	Loss: 0.638507
Train Epoch: 2 [49920/60032 (83%)]	Loss: 0.453861
Train Epoch: 2 [53760/60032 (90%)]	Loss: 0.535925
Train Epoch: 2 [57600/60032 (96%)]	Loss: 0.244492

Test set: Average loss: 0.3281, Accuracy: 9063/10000 (91%)

Train Epoch: 3 [30720/60032 (51%)]	Loss: 0.395385
Train Epoch: 3 [34560/60032 (58%)]	Loss: 0.261490
Train Epoch: 3 [49920/60032 (83%)]	Loss: 0.350926
Train Epoch: 3 [53760/60032 (90%)]	Loss: 0.268865
Train Epoch: 3 [57600/60032 (96%)]	Loss: 0.332211



Test set: Average loss: 0.2733, Accuracy: 9251/10000 (93%)

Running with N=3, X=5
Train Epoch: 1 [7680/60032 (13%)]	Loss: 2.256382
Train Epoch: 1 [11520/60032 (19%)]	Loss: 2.065276
Train Epoch: 1 [15360/60032 (26%)]	Loss: 1.688400
Train Epoch: 1 [30720/60032 (51%)]	Loss: 1.179197
Train Epoch: 1 [34560/60032 (58%)]	Loss: 0.749203
Train Epoch: 1 [42240/60032 (70%)]	Loss: 0.580598
Train Epoch: 1 [46080/60032 (77%)]	Loss: 0.663559
Train Epoch: 1 [49920/60032 (83%)]	Loss: 0.504522
Train Epoch: 1 [53760/60032 (90%)]	Loss: 0.425321

Test set: Average loss: 0.3872, Accuracy: 8912/10000 (89%)

Train Epoch: 2 [7680/60032 (13%)]	Loss: 0.364883
Train Epoch: 2 [11520/60032 (19%)]	Loss: 0.323480
Train Epoch: 2 [15360/60032 (26%)]	Loss: 0.278147
Train Epoch: 2 [30720/60032 (51%)]	Loss: 0.304774
Train Epoch: 2 [34560/60032 (58%)]	Loss: 0.469818
Train Epoch: 2 [42240/60032 (70%)]	Loss: 0.339882
Train Epoch: 2 [46080/60032 (77%)]	Loss: 0.289331
Train Epoch: 2 [49920/60032 (83%)]	Loss: 0.263013
Train Ep


Test set: Average loss: 0.1899, Accuracy: 9436/10000 (94%)

Running with N=3, X=7
Train Epoch: 1 [0/60032 (0%)]	Loss: 2.306076
Train Epoch: 1 [3840/60032 (6%)]	Loss: 2.161293
Train Epoch: 1 [15360/60032 (26%)]	Loss: 1.686235
Train Epoch: 1 [19200/60032 (32%)]	Loss: 1.121563
Train Epoch: 1 [23040/60032 (38%)]	Loss: 0.573718
Train Epoch: 1 [26880/60032 (45%)]	Loss: 0.601819
Train Epoch: 1 [38400/60032 (64%)]	Loss: 0.578173
Train Epoch: 1 [49920/60032 (83%)]	Loss: 0.452109
Train Epoch: 1 [53760/60032 (90%)]	Loss: 0.382749
Train Epoch: 1 [57600/60032 (96%)]	Loss: 0.259029

Test set: Average loss: 0.3239, Accuracy: 9020/10000 (90%)

Train Epoch: 2 [0/60032 (0%)]	Loss: 0.352349
Train Epoch: 2 [3840/60032 (6%)]	Loss: 0.193373
Train Epoch: 2 [15360/60032 (26%)]	Loss: 0.233092
Train Epoch: 2 [19200/60032 (32%)]	Loss: 0.359445
Train Epoch: 2 [23040/60032 (38%)]	Loss: 0.207951
Train Epoch: 2 [26880/60032 (45%)]	Loss: 0.225892
Train Epoch: 2 [38400/60032 (64%)]	Loss: 0.308954
Train Epoch: 2 [4992


Test set: Average loss: 0.1434, Accuracy: 9590/10000 (96%)

Running with N=3, X=10
Train Epoch: 1 [0/60032 (0%)]	Loss: 2.306076
Train Epoch: 1 [3840/60032 (6%)]	Loss: 2.161293
Train Epoch: 1 [7680/60032 (13%)]	Loss: 1.911803
Train Epoch: 1 [11520/60032 (19%)]	Loss: 1.265940
Train Epoch: 1 [15360/60032 (26%)]	Loss: 0.804201
Train Epoch: 1 [19200/60032 (32%)]	Loss: 0.696759
Train Epoch: 1 [23040/60032 (38%)]	Loss: 0.362693
Train Epoch: 1 [26880/60032 (45%)]	Loss: 0.510118
Train Epoch: 1 [30720/60032 (51%)]	Loss: 0.441816
Train Epoch: 1 [34560/60032 (58%)]	Loss: 0.359350
Train Epoch: 1 [38400/60032 (64%)]	Loss: 0.459955
Train Epoch: 1 [42240/60032 (70%)]	Loss: 0.256071
Train Epoch: 1 [46080/60032 (77%)]	Loss: 0.427629
Train Epoch: 1 [49920/60032 (83%)]	Loss: 0.369672
Train Epoch: 1 [53760/60032 (90%)]	Loss: 0.300360
Train Epoch: 1 [57600/60032 (96%)]	Loss: 0.212879

Test set: Average loss: 0.2595, Accuracy: 9223/10000 (92%)

Train Epoch: 2 [0/60032 (0%)]	Loss: 0.292925
Train Epoch: 2 [38


Test set: Average loss: 0.1069, Accuracy: 9690/10000 (97%)

Problem 2b.
Running with X=3, N=3
Train Epoch: 1 [15360/60032 (26%)]	Loss: 2.189415
Train Epoch: 1 [38400/60032 (64%)]	Loss: 1.894555
Train Epoch: 1 [42240/60032 (70%)]	Loss: 1.259834
Train Epoch: 1 [46080/60032 (77%)]	Loss: 0.887583

Test set: Average loss: 0.6186, Accuracy: 8551/10000 (86%)

Train Epoch: 2 [15360/60032 (26%)]	Loss: 0.590432
Train Epoch: 2 [38400/60032 (64%)]	Loss: 0.515363
Train Epoch: 2 [42240/60032 (70%)]	Loss: 0.482194
Train Epoch: 2 [46080/60032 (77%)]	Loss: 0.388739

Test set: Average loss: 0.3951, Accuracy: 8729/10000 (87%)

Train Epoch: 3 [15360/60032 (26%)]	Loss: 0.184496


KeyboardInterrupt: 

In [17]:
print("Problem 2b.")
n_vals = [3, 5, 10]
for n in n_vals:
    print(f"Running with X=5, N={n}")
    run(N=n, X=5)

Problem 2b.
Running with X=5, N=3
Train Epoch: 1 [0/60032 (0%)]	Loss: 2.306076
Train Epoch: 1 [3840/60032 (6%)]	Loss: 2.161293
Train Epoch: 1 [26880/60032 (45%)]	Loss: 1.771049
Train Epoch: 1 [30720/60032 (51%)]	Loss: 1.143258
Train Epoch: 1 [34560/60032 (58%)]	Loss: 0.740875
Train Epoch: 1 [38400/60032 (64%)]	Loss: 0.672293
Train Epoch: 1 [42240/60032 (70%)]	Loss: 0.478192
Train Epoch: 1 [46080/60032 (77%)]	Loss: 0.561012

Test set: Average loss: 0.3793, Accuracy: 8945/10000 (89%)

Train Epoch: 2 [0/60032 (0%)]	Loss: 0.394081
Train Epoch: 2 [3840/60032 (6%)]	Loss: 0.244425
Train Epoch: 2 [26880/60032 (45%)]	Loss: 0.341500
Train Epoch: 2 [30720/60032 (51%)]	Loss: 0.308609
Train Epoch: 2 [34560/60032 (58%)]	Loss: 0.465418
Train Epoch: 2 [38400/60032 (64%)]	Loss: 0.369204
Train Epoch: 2 [42240/60032 (70%)]	Loss: 0.323698
Train Epoch: 2 [46080/60032 (77%)]	Loss: 0.292309

Test set: Average loss: 0.2762, Accuracy: 9122/10000 (91%)

Train Epoch: 3 [0/60032 (0%)]	Loss: 0.311858
Train Epoch: 


Test set: Average loss: 0.1860, Accuracy: 9437/10000 (94%)

Running with X=5, N=5
Train Epoch: 1 [7680/60032 (13%)]	Loss: 2.256382
Train Epoch: 1 [11520/60032 (19%)]	Loss: 2.065276
Train Epoch: 1 [26880/60032 (45%)]	Loss: 1.773756
Train Epoch: 1 [30720/60032 (51%)]	Loss: 1.145211
Train Epoch: 1 [34560/60032 (58%)]	Loss: 0.740973
Train Epoch: 1 [42240/60032 (70%)]	Loss: 0.572766
Train Epoch: 1 [46080/60032 (77%)]	Loss: 0.657109
Train Epoch: 1 [49920/60032 (83%)]	Loss: 0.502181
Train Epoch: 1 [53760/60032 (90%)]	Loss: 0.425289

Test set: Average loss: 0.3873, Accuracy: 8919/10000 (89%)

Train Epoch: 2 [7680/60032 (13%)]	Loss: 0.364866
Train Epoch: 2 [11520/60032 (19%)]	Loss: 0.324169
Train Epoch: 2 [26880/60032 (45%)]	Loss: 0.334088
Train Epoch: 2 [30720/60032 (51%)]	Loss: 0.311561
Train Epoch: 2 [34560/60032 (58%)]	Loss: 0.463665
Train Epoch: 2 [42240/60032 (70%)]	Loss: 0.334762
Train Epoch: 2 [46080/60032 (77%)]	Loss: 0.296561
Train Epoch: 2 [49920/60032 (83%)]	Loss: 0.263956
Train Ep


Test set: Average loss: 0.1281, Accuracy: 9601/10000 (96%)

Running with X=5, N=10
Train Epoch: 1 [7680/60032 (13%)]	Loss: 2.256382
Train Epoch: 1 [11520/60032 (19%)]	Loss: 2.065276
Train Epoch: 1 [19200/60032 (32%)]	Loss: 1.958374
Train Epoch: 1 [23040/60032 (38%)]	Loss: 1.339540
Train Epoch: 1 [26880/60032 (45%)]	Loss: 0.877905
Train Epoch: 1 [42240/60032 (70%)]	Loss: 0.585238
Train Epoch: 1 [46080/60032 (77%)]	Loss: 0.655941
Train Epoch: 1 [57600/60032 (96%)]	Loss: 0.348908

Test set: Average loss: 0.4004, Accuracy: 8749/10000 (87%)

Train Epoch: 2 [7680/60032 (13%)]	Loss: 0.365334
Train Epoch: 2 [11520/60032 (19%)]	Loss: 0.326093
Train Epoch: 2 [19200/60032 (32%)]	Loss: 0.464138
Train Epoch: 2 [23040/60032 (38%)]	Loss: 0.289958
Train Epoch: 2 [26880/60032 (45%)]	Loss: 0.303797
Train Epoch: 2 [42240/60032 (70%)]	Loss: 0.364029
Train Epoch: 2 [46080/60032 (77%)]	Loss: 0.305054
Train Epoch: 2 [57600/60032 (96%)]	Loss: 0.156041

Test set: Average loss: 0.2449, Accuracy: 9279/10000 (93